In [1]:
import h5py
import numpy as np
import os
from queue import Queue
import random
from IPython.display import clear_output

In [2]:
colors = []
games = []
cnt = 0
def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        print(f"colors = {colors[cnt]}")
        df = file.read()
        # 如果有讓子，跳過
        if df[5] != '0' and df[5] != '1':
            return
        game = df.split(';')
        move_list = game[2:]
        games.append(move_list)

In [3]:
def is_sgf_file(file_name):
    # 使用 os.path.splitext 取得檔案擴展名
    _, extension = os.path.splitext(file_name)
    # 判斷擴展名是否為 ".sgf"
    return extension.lower() == '.sgf'

In [4]:
chars = 'abcdefghijklmnopqrs'
coordinates = {k:v for v,k in enumerate(chars)}
chartonumbers = {k:v for k,v in enumerate(chars)}
chars_reverse = 'srqponmlkjihgfedcba'
coordinates_reverse = {k:v for v,k in enumerate(chars_reverse)}
coordinates

{'a': 0,
 'b': 1,
 'c': 2,
 'd': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'j': 9,
 'k': 10,
 'l': 11,
 'm': 12,
 'n': 13,
 'o': 14,
 'p': 15,
 'q': 16,
 'r': 17,
 's': 18}

# Feature Map(共19個)
    0:初段棋手的盤面
    1:對手盤面
    2~5:初段棋手的氣數 (1, 2, 3, >=4)
    6~9:對手的氣數(1, 2, 3, >=4)
    10:標示空地
    11~18:最後八步

In [5]:
player_board = 0
oppnent_board = 1
player_air_1 = 2
player_air_2 = 3
player_air_3 = 4
player_air_4 = 5
oppnent_air_1 = 6
oppnent_air_2 = 7
oppnent_air_3 = 8
oppnent_air_4 = 9
empty_board = 10
last_1 = 11
last_8 = 18

In [6]:
liber = Queue()
def count_air2(move):
    bfs = []
    col = coordinates[move[2]]
    row = coordinates[move[3]]
    #找四周要bfs的
    now_color = -1
    if player[row][col] == 1:
        now_color = 1
    else:
        now_color = 0
    if col - 1 >= 0 and x[row][col - 1][empty_board] == 0:
        if player[row][col - 1] != now_color:
            bfs.insert(0, [row, col - 1])
        else:
            bfs.append([row, col - 1])
    if col + 1 < 19 and x[row][col + 1][empty_board] == 0:
        if player[row][col + 1] != now_color:
            bfs.insert(0, [row, col + 1])
        else:
            bfs.append([row, col + 1])
    if row - 1 >= 0 and x[row - 1][col][empty_board] == 0:
        if player[row - 1][col] != now_color:
            bfs.insert(0, [row - 1, col])
        else:
            bfs.append([row - 1, col])
    if row + 1 < 19 and x[row + 1][col][empty_board] == 0:
        if player[row + 1][col] != now_color:
            bfs.insert(0, [row + 1, col])
        else:
            bfs.append([row + 1, col])

#     if col - 1 >= 0 and x[row][col - 1][empty_board] == 0:
#         bfs.append([row, col - 1])
#     if col + 1 < 19 and x[row][col + 1][empty_board] == 0:
#         bfs.append([row, col + 1])
#     if row - 1 >= 0 and x[row - 1][col][empty_board] == 0:
#         bfs.append([row - 1, col])
#     if row + 1 < 19 and x[row + 1][col][empty_board] == 0:
#         bfs.append([row + 1, col])
            
    bfs.append([row, col])
    for m in bfs:
        BFS(m)
    bfs = []
    if not liber.empty():
        while not liber.empty():
            li_row, li_col = liber.get()
            if li_col - 1 >= 0 and x[li_row][li_col - 1][empty_board] == 0:
                bfs.append([li_row, li_col - 1])
            if li_col + 1 < 19 and x[li_row][li_col + 1][empty_board] == 0:
                bfs.append([li_row, li_col + 1])
            if li_row - 1 >= 0 and x[li_row - 1][li_col][empty_board] == 0:
                bfs.append([li_row - 1, li_col])
            if li_row + 1 < 19 and x[li_row + 1][li_col][empty_board] == 0:
                bfs.append([li_row + 1, li_col])
        for m in bfs:
            BFS(m)
        
def BFS(now):
    q = Queue()
    row = now[0]
    col = now[1]
    q.put((row, col))
    visited = [[False for _ in range(19)] for _ in range(19)]
    nodes = []
    air = 0
    next_board = player_air_1
    board = player
    if oppnent[row, col] == 1:
        board = oppnent
        next_board = oppnent_air_1
    while not q.empty():
        m, n = q.get()
        if m < 0 or n < 0 or m >= 19 or n >= 19 or visited[m][n]:
            continue
        visited[m][n] = True
        if board[m, n] == 1:
            nodes.append([m, n])
            q.put((m - 1, n))
            q.put((m + 1, n))
            q.put((m, n - 1))
            q.put((m, n + 1))
        elif x[m, n, empty_board] == 1: #空地 -> 自由度+1
            air += 1
    dele = -1
    for i in range(player_air_1, player_air_1 + 8):
        if x[row][col][i] == 1:
            dele = i
            break
    air = min(air, 4)
    for node in nodes:
        n_row = node[0]
        n_col = node[1]
        #原本的改0，現在的氣數對應的棋盤改1
        for i in range(player_air_1, player_air_1 + 8):
            x[n_row][n_col][i] = 0;
        if air == 0:
            x[n_row][n_col][empty_board] = 1
            player[n_row, n_col] = 0
            oppnent[n_row, n_col] = 0
            liber.put(node)
        else:
            x[n_row][n_col][next_board + air - 1] = 1

In [7]:
x = np.zeros((19,19,19))
player = np.zeros((19, 19))
oppnent = np.zeros((19, 19))


def prepare_input(moves, player_color):
    sz = len(moves)
    if sz == 0:
        return x;
    move = moves[-1]
    
    color = move[0]
    column = coordinates[move[2]]
    row = coordinates[move[3]]
#     global player
    if color == player_color: 
#         print("player in")
#         for j in range(19):
#             for k in range(19):
#                 print((int)(player[j][k]), end = ' ')
#             print()
#         print()
        player[row,column] = 1
#         print(row,column)
#         for j in range(19):
#             for k in range(19):
#                 print((int)(player[j][k]), end = ' ')
#             print()
#         print()
    else: 
        oppnent[row,column] = 1
    x[row,column,empty_board] = 0
    
    #倒數8步
    for i in range(last_8, last_1, -1):
        x[:, :, i] = x[:, :, i - 1]
    x[:, :, last_1] = 0
    x[row, column, last_1] = 1
    count_air2(move)
    
    x[:, :, player_board] = player.copy()
    x[:, :, oppnent_board] = oppnent.copy()
    
    
    
    return x

def prepare_label(move):
    column = coordinates[move[2]]
    row = coordinates[move[3]]
    return column*19+row

def prepare_label_rotate90(move):
    column = coordinates[move[2]]
    row = coordinates_reverse[move[3]]
    return row*19+column

In [9]:
hdf5 = h5py.File('./1_100/hdf5/test.hdf5', 'a')
def creat_feature_map(people):
    
#     if os.path.isfile('./1_100/hdf5/{people}.hdf5'):
#         os.remove('./1_100/hdf5/{people}.hdf5')
#     hdf5 = h5py.File('./1_100/hdf5/{people}.hdf5', 'a')
    global hdf5
    hdf5.create_dataset('Xs', (0, 19, 19, 19), maxshape=(None, 19, 19, 19), chunks=(10, 19, 19, 19)
                        ,dtype=np.float32, compression="lzf")
    hdf5.create_dataset('Ys', (0, 1), maxshape=(None, 1)
                        ,dtype=np.float32, compression="lzf")
    x_dataset = hdf5['Xs']
    y_dataset = hdf5['Ys']

    x_batch = [] # Initialize data batch
    y_batch = []
    batch_size = 1
    print(f'total{len(games)}')
    global x
    for game_i, game in enumerate(games): # Iterate through games
#             clear_output(wait=True)
            print(game_i)
            x = np.zeros((19,19,19))
#             for i in range(10, 11):
#                 for j in range(19):
#                     for k in range(19):
#                         print((int)(x[j][k][i]), end = ' ')
#                     print()
#                 print()
            global player
            global oppnent
            player = np.zeros((19, 19))
            oppnent = np.zeros((19, 19))
            x[:, :, empty_board] = np.ones((19, 19))
            moves_list = game
            for count, move in enumerate(moves_list):
#                 print(type(colors[game_i]))
                pi = prepare_input(moves_list[:count], colors[game_i])
                if colors[game_i] == move[0]:
#                     print(moves_list[:count])
#                     for i in range(0, 19):
#                         for j in range(19):
#                             for k in range(19):
#                                 print((int)(pi[j][k][i]), end = ' ')
#                             print()
#                         print()
#                     for j in range(19):
#                         for k in range(19):
#                             print((int)(player[j][k]), end = ' ')
#                         print()
#                     print()
                    x_batch.append(pi.copy())
                    y_batch.append([prepare_label(moves_list[count])])
                    if len(x_batch) == batch_size or (game_i == len(games) - 1 
                                            and (count == len(moves_list) - 1 or count == len(moves_list) - 2)): # Yield when reached batch size
                            x_dataset.resize(x_dataset.shape[0] + len(x_batch), axis = 0)
                            x_dataset[-len(x_batch):] = np.array(x_batch)
                            y_dataset.resize(y_dataset.shape[0] + len(y_batch), axis = 0)
                            y_dataset[-len(y_batch):] = np.array(y_batch)
                            x_batch = []
                            y_batch = []
    print(f"name : {people}")
    for idx, arr in enumerate(x_dataset):
        print(f"idx : {idx}")
        for i in range(0, 2):
            for j in range(19):
                for k in range(19):
                    print((int)(arr[j][k][i]), end = ' ')
                print()
            print()
    hdf5.close()

In [10]:
folder_path = './1_100/test/'  # 替換為實際的資料夾路徑

# test -> 人名 -> B/W -> 棋譜

# 確保資料夾存在
if os.path.exists(folder_path) and os.path.isdir(folder_path):
    # 列舉資料夾中的所有檔案
    for people in os.listdir(folder_path):
        colors = []
        games = []
        cnt = 0
        print(f"people name {people}")
        people_path = os.path.join(folder_path, people)
        
        if os.path.exists(people_path) and os.path.isdir(people_path):
            for player_color in os.listdir(people_path):
                print(f"color {player_color}")
                color_path = os.path.join(people_path, player_color)
                if os.path.exists(color_path) and os.path.isdir(color_path):
                    for filename in os.listdir(color_path):
                        if is_sgf_file(filename):
                            print(f"file name {filename}")
                            print("in")
                            print(f"player color : {type(player_color[0])}")
                            colors.append(player_color[0])
                            file_path = os.path.join(color_path, filename)
                            read_file(file_path)
                            cnt += 1
        print(colors)
        global hdf5
        if os.path.isfile(f'./1_100/hdf5/{people}.hdf5'):
            os.remove(f'./1_100/hdf5/{people}.hdf5')
        hdf5 = h5py.File(f'./1_100/hdf5/{people}.hdf5', 'a')
        creat_feature_map(people)
else:
    print(f"The folder {folder_path} does not exist or is not a directory.")


# print(games)

people name 123
color B
file name 44219163.sgf
in
player color : <class 'str'>
colors = B
file name 44219326.sgf
in
player color : <class 'str'>
colors = B
color W
file name 44220848.sgf
in
player color : <class 'str'>
colors = W
['B', 'B', 'W']
total3
0
1
2
name : 123
idx : 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 

idx : 24
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 
0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 
0 1 0 1 1 0 0 1 0 1 1 0 0 1 0 0 0 0 0 
0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 1 1 0 0 0 1 0 0 0 1 0 0 1 1 0 0 0 
0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 
0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 
0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0

idx : 127
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 
0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 1 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 
0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 1 1 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 
0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 
0 0 0 0 0 0 0 

idx : 223
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 

idx : 319
0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 
0 0 1 0 1 0 0 0 0 1 1 1 0 1 1 1 1 1 0 
0 0 0 0 1 1 0 0 1 1 1 1 1 0 0 0 1 0 0 
0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 
0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 
0 0 0 0 0 0 1 1 1 0 0 0 1 0 0 0 0 1 0 
0 0 0 0 0 1 0 1 1 0 0 0 1 0 0 0 0 1 0 
0 0 0 0 0 0 0 0 1 1 1 1 0 1 0 0 1 1 0 
0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0 1 1 
0 0 0 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 1 
0 0 0 0 0 0 0 0 0 1 0 1 0 1 1 1 0 1 0 
0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0 1 1 0 
0 1 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 1 0 
0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 
0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1 0 
0 0 1 0 0 0 1 1 0 1 0 1 0 0 0 0 1 1 0 
0 0 0 1 1 0 1 0 1 1 0 1 1 0 0 1 1 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

0 0 0 0 1 1 0 1 1 1 1 1 0 0 0 0 0 0 0 
0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 1 
0 0 0 0 0 0 1 1 0 0 0 0 0 1 1 1 0 1 0 
0 0 0 0 0 1 1 0 0 1 0 1 1 0 0 1 1 0 0 
0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 0 
0 0 1 1 0 1 1 0 0 0 1 0 1 0 0 0 0 1 0 
0 1 0 0 0 1 0 

2
name : 456
idx : 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0

idx : 41
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 
0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 
0 1 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 
0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 1 1 0 0 
0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 
0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 
0 0 1 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 
0 1 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 
0 0 0 0 1 1 0 0 0 0 0 1 1 1 0 1 0 0 0 
0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 
0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 
0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 
0 0 0 0 0 0 0 0

idx : 141
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 
0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 
0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 
0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 
0 1 0 0 1 1 0 0 0 1 1 0 0 1 0 0 0 0 0 
0 1 1 1 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 
0 0 0 0 1 0 0 0 0 0 1 1 1 0 0 0 0 0 0 
0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 
0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 

idx : 240
0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 
0 0 0 0 0 1 1 0 0 1 0 0 0 0 1 0 0 1 0 
0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 0 1 
0 0 0 0 1 0 0 1 0 1 1 0 0 1 0 1 1 1 0 
0 0 0 1 0 0 0 0 1 1 0 0 0 1 1 0 0 0 0 
1 0 1 1 1 1 0 1 0 0 1 1 1 0 0 0 0 0 0 
0 1 1 0 1 0 1 1 1 0 0 1 0 1 0 0 0 0 0 
0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 
0 0 1 0 0 1 1 1 1 0 0 0 0 0 1 1 0 0 0 
0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 
0 0 1 0 0 1 0 0 1 0 1 0 0 0 1 0 1 0 0 
0 0 0 0 0 1 0 0 0 1 1 1 0 0 0 1 0 0 0 
0 0 0 0 1 1 0 0 0 1 0 1 1 0 0 0 1 0 0 
0 0 0 0 0 1 0 1 1 0 0 0 0 1 0 0 1 0 0 
1 0 0 1 1 1 1 0 0 0 0 0 0 1 0 0 1 0 1 
1 1 0 0 1 1 0 0 0 1 1 1 0 1 0 0 0 1 0 
0 1 1 1 0 0 0 0 1 0 0 1 0 0 0 0 0 1 0 
0 1 0 0 1 0 0 1 1 0 1 1 1 0 0 0 0 0 0 
0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 

0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 1 0 
0 0 0 0 1 0 0 0 0 0 1 1 1 0 0 1 1 0 0 
1 1 1 1 1 0 1 0 0 0 0 1 0 1 1 1 0 0 0 
0 0 0 1 0 1 1 0 0 0 0 1 1 0 1 0 0 0 0 
1 1 1 0 0 0 1 1 0 0 1 1 0 0 0 0 0 0 0 
0 1 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 
0 0 0 1 0 1 0 

idx : 342
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 
0 0 1 0 0 0 0 0 1 0 1 1 0 0 0 0 0 1 0 
1 0 1 0 0 0 0 0 0 1 0 0 0 0 1 1 1 0 0 
0 1 0 1 0 0 0 0 0 0 1 1 1 0 0 1 0 0 0 
0 0 0 0 1 0 0 0 0 1 0 0 1 1 1 0 1 0 0 
0 0 0 0 1 0 1 1 0 0 0 0 0 0 1 0 0 0 0 
0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 1 0 
0 0 0 1 1 0 1 1 1 1 1 0 1 0 0 1 1 1 0 
0 0 0 0 1 1 0 0 0 0 1 0 1 0 0 0 1 0 0 
0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 0 
0 0 0 0 1 0 0 0 1 1 1 0 0 0 1 1 0 0 0 
0 0 1 1 1 1 0 0 0 1 0 1 0 0 0 0 1 0 0 
0 1 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 
0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 
0 0 1 0 0 0 0 0 1 1 1 1 1 0 0 1 0 0 0 
0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 
0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 
0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 1 1 0 0 
0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 0 1 0 
0 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 1 
0 1 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 
0 0 1 1 0 0 0 

In [11]:
hdf5.close()